#### Transformation of data using custom transformation functions using pyTorch

In [1]:
# Using Dataset and composed transforms to  transforma data in PyTorch 
import torch
import numpy as np
from torch.utils.data import Dataset

In [122]:
# custom dataset creation

class CustomDataset(Dataset):

    def __init__(self, transform = None):
        super().__init__()    
        
        self.xy = np.loadtxt('./Data/wine.csv', skiprows = 1, delimiter = ",", dtype = np.float32)
        self.x = self.xy[: , 1:]
        self.y = self.xy[: , [0]]
        self.transform = transform

    def __getitem__(self, index):
        data = self.x[index], self.y[index]
        
        if self.transform:
            data = self.transform(data)
        
        return data
    
    def __len__(self):
        return self.xy.shape[0]

In [123]:
# Implementing Custom Transform Function
class ToTensor:
    def __call__(self, data):
        inputs, outputs =  data
        return torch.from_numpy(inputs), torch.from_numpy(outputs)

In [125]:
# converting the numpy array to tensor using the custom transform function
wine = CustomDataset(transform=ToTensor())
wine[0]

(tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
         3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
         1.0650e+03]),
 tensor([1.]))

In [138]:
# Implementing Composed Transform Function 

from dataclasses import dataclass
import torch

@dataclass
class ArithmeticTransform:
    operations: str
    num: int

    def __call__(self, data: tuple[torch.Tensor, torch.Tensor]) -> tuple[torch.Tensor, torch.Tensor]:
        inputs, outputs = data

        operation_dict = {
            '*': torch.mul,
            '+': torch.add,
            '-': torch.sub,
            '/': torch.div
        }

        operation_func = operation_dict.get(self.operations)
        if operation_func:
            inputs = operation_func(inputs, self.num)
        
        return inputs, outputs

In [140]:
import torchvision

"""
Main use case of this Compose from torchvision is to compose several transforms together.
Here we are using it to compose our custom transform functions of ToTensor and ArithmeticTransform.
"""

# This composed transform will first convert the numpy array to tensor and then multiply the input tensor by 2
composed = torchvision.transforms.Compose([ToTensor(),ArithmeticTransform('*',2)])
dataset = CustomDataset(transform=composed)
dataset[0]

(tensor([2.8460e+01, 3.4200e+00, 4.8600e+00, 3.1200e+01, 2.5400e+02, 5.6000e+00,
         6.1200e+00, 5.6000e-01, 4.5800e+00, 1.1280e+01, 2.0800e+00, 7.8400e+00,
         2.1300e+03]),
 tensor([1.]))